```python
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
```

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
import torch.nn as nn
import bitsandbytes as bnb

from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

True
Tesla P40


### Load adapters from the Hub

**Note**: *The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.*

In [2]:
from peft import LoraConfig, get_peft_model
from peft import PeftModel, PeftConfig

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

In [4]:
peft_model_id = "ybelkada/opt-6.7b-lora"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    load_in_8bit=True, # load_in_4bit
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model (7GB)
model = PeftModel.from_pretrained(model, peft_model_id).to(device)


/home/loc/miniconda3/envs/py3x/lib/python3.10/site-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['bias', 'enable_lora', 'fan_in_fan_out', 'lora_alpha', 'lora_dropout', 'merge_weights', 'r', 'target_modules'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/loc/miniconda3/envs/py3x/lib/python3.10/site-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['enable_lora', 'merge_weights'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


In [7]:
batch = tokenizer("Machine learning is: ", return_tensors='pt').to(device)

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=512)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/tmp/ipykernel_23939/4272391302.py:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():




 Machine learning is:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
